

## Make a questionnaire that asks questions regarding 

-- Use-ases for shortcuts to assign macros to either mouse buttons or keybaord keys



In [1]:
import ollama
import streamlit
import pandas as pd
import numpy as np
import pydantic
import datetime
import time
import os

## Testing ollama generations

In [2]:
stream = ollama.chat(
    model = "gemma2:2b",
    messages= [
        {"role":"user", "content":"Hello, how are you?"}
    ],
    stream=True
)
for s in stream:
    print(s['message']['content'], end="", flush=True)


I'm doing well! Thanks for asking. 😊  

How can I help you today? 


## Defining the functions to do RAG on for further questioning and recommendations

In [8]:
functions = [
    {"name": "add_numbers", "description": "Adds two numbers and returns the result."},
    {"name": "subtract_numbers", "description": "Subtracts the second number from the first."},
    {"name": "multiply_numbers", "description": "Multiplies two numbers and returns the result."},
    {"name": "divide_numbers", "description": "Divides the first number by the second, returns the quotient."}
]


from sentence_transformers import SentenceTransformer
import numpy as np

# Load pre-trained embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode function descriptions
function_embeddings = {f["name"]: model.encode(f["description"]) for f in functions}

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\arvin\anaconda3\envs\mnlp-a2\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\arvin\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

c:\Users\arvin\anaconda3\envs\mnlp-a2\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [14]:
from sklearn.metrics.pairwise import cosine_similarity

def retrieve_function(query):
    query_embedding = model.encode(query)
    similarities = {name: cosine_similarity([query_embedding], [embedding])[0][0]
                    for name, embedding in function_embeddings.items()}
    best_match = max(similarities, key=similarities.get)
    return best_match, similarities[best_match]

def generate_response(query):
    function_name, confidence = retrieve_function(query)
    return f"The most relevant function is '{function_name}'. Confidence: {confidence:.2f}"



query = "What is 4/2?"
response = generate_response(query)
print(response)


The most relevant function is 'subtract_numbers'. Confidence: 0.37
